# Example: scenarios for TAMU or RTS grids


In [1]:
import pandas as pd
import numpy as np
import time

from powerscenarios.parser import Parser
from powerscenarios.grid import Grid

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)

# plotting (optional)
import cufflinks as cl

cl.go_offline()


# choose grid

# parse TAMU grid .aux files

In [3]:
# choose TAMU grid
# grid files can be downloaded from:
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/

grid_name = "ACTIVSg200"  # TAMU 200 bus case
# grid_name = "ACTIVSg2000"  # TAMU 2000 bus case
# grid_name = 'ACTIVSg10k' # TAMU 10000 bus case

# path to .aux file (TAMU grids) obtained from e.g.
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/activsg200/
data_dir = "../data/grid-data/"
aux_file_name = data_dir + grid_name + "/" + grid_name + ".aux"

# parse original .aux file and return dataframes for buses, generators, and wind generators
# here, we need .aux files because those are the only ones with Latitute/Longitude information
parser = Parser()
bus_df, gen_df, wind_gen_df = parser.parse_tamu_aux(aux_file_name)

# see what you got
print("bus_df:")
bus_df.head()
print("gen_df:")
gen_df.head()
print("wind_gen_df:")
wind_gen_df.head()

bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6


gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.53,1.36,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.53,1.36,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.53,1.36,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.53,1.36,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
4,53,1,9.07,2.72,1.0,Coal,53_Coal_1,RANTOUL25,40.312222,-88.159444,6


wind_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7


<IPython.core.display.Javascript object>

# or parse RTS grid .csv files

In [5]:
grid_name = "RTS"

data_dir = "../data"
bus_csv_filename = data_dir + "/grid-data/" + grid_name + "/bus.csv"
gen_csv_filename = data_dir + "/grid-data/" + grid_name + "/gen.csv"

parser = Parser()

# if solar2wind, will replace all solar to wind
bus_df, gen_df, wind_gen_df = parser.parse_rts_csvs(
    bus_csv_filename, gen_csv_filename, solar2wind=True
)

# see what you got
print("bus_df:")
bus_df.head()
print("gen_df:")
gen_df.head()
print("wind_gen_df:")
wind_gen_df.head()

bus_df:


,BusNum,Bus Name,BaseKV,Bus Type,MW Load,MVAR Load,V Mag,V Angle,MW Shunt G,MVAR Shunt B,Area,Sub Area,Zone,Latitude,Longitude
0,101,Abel,138.0,PV,108.0,22.0,1.04777,-7.74152,0.0,0.0,1,11.0,11.0,33.396103,-113.835642
1,102,Adams,138.0,PV,97.0,20.0,1.04783,-7.81784,0.0,0.0,1,11.0,12.0,33.357678,-113.825933
2,103,Adler,138.0,PQ,180.0,37.0,1.01085,-7.21090,0.0,0.0,1,11.0,11.0,33.536833,-114.670399
3,104,Agricola,138.0,PQ,74.0,15.0,1.01765,-10.56614,0.0,0.0,1,11.0,11.0,33.812304,-113.825419
4,105,Aiken,138.0,PQ,71.0,14.0,1.03568,-10.70887,0.0,0.0,1,11.0,11.0,33.659560,-113.999023


gen_df:


,GenUID,BusNum,Gen ID,Unit Group,Unit Type,Category,GenFuelType,MW Inj,MVAR Inj,V Setpoint p.u.,GenMWMax,PMin MW,QMax MVAR,QMin MVAR,Min Down Time Hr,Min Up Time Hr,Ramp Rate MW/Min,Start Time Cold Hr,Start Time Warm Hr,Start Time Hot Hr,Start Heat Cold MBTU,Start Heat Warm MBTU,Start Heat Hot MBTU,Non Fuel Start Cost $,FOR,MTTF Hr,MTTR Hr,Scheduled Maint Weeks,Fuel Price $/MMBTU,Output_pct_0,Output_pct_1,Output_pct_2,Output_pct_3,HR_avg_0,HR_incr_1,HR_incr_2,HR_incr_3,Fuel Sulfur Content %,Emissions SO2 Lbs/MMBTU,Emissions NOX Lbs/MMBTU,Emissions Part Lbs/MMBTU,Emissions CO2 Lbs/MMBTU,Emissions CH4 Lbs/MMBTU,Emissions N2O Lbs/MMBTU,Emissions CO Lbs/MMBTU,Emissions VOCs Lbs/MMBTU,Damping Ratio,Inertia MJ/MW,Base MVA,Transformer X p.u.,Unit X p.u.,Pump Load MW,Storage Roundtrip Efficiency
0,101_CT_1,101,1,U20,CT,Oil CT,Oil,8.0,4.96,1.0468,20.0,8,10,0,1.0,1.0,3.0,1,0.0,0.0,5.0,5.0,5.0,0,0.10,450,50,2.0,10.34940,0.400000,0.600000,0.800000,1,13114,9456,9476,10352,0.2,0.2,0.5,0.036,160,0.002,0.004,0.11,0.040,0,2.8,24,0.13,0.32,0,0
1,101_CT_2,101,2,U20,CT,Oil CT,Oil,8.0,4.96,1.0468,20.0,8,10,0,1.0,1.0,3.0,1,0.0,0.0,5.0,5.0,5.0,0,0.10,450,50,2.0,10.34940,0.400000,0.600000,0.800000,1,13114,9456,9476,10352,0.2,0.2,0.5,0.036,160,0.002,0.004,0.11,0.040,0,2.8,24,0.13,0.32,0,0
2,101_STEAM_3,101,3,U76,STEAM,Coal,Coal,76.0,0.14,1.0468,76.0,30,30,-25,4.0,8.0,2.0,12,10.0,3.0,5284.8,4861.4,3379.4,0,0.02,1960,40,3.0,2.11399,0.394737,0.596491,0.798246,1,13270,6713,8028,8549,Unit-specific,Unit-specific,Unit-specific,Unit-specific,210,0.001,0.004,0.02,0.003,0,3.0,89,0.13,0.30,0,0
3,101_STEAM_4,101,4,U76,STEAM,Coal,Coal,76.0,0.14,1.0468,76.0,30,30,-25,4.0,8.0,2.0,12,10.0,3.0,5284.8,4861.4,3379.4,0,0.02,1960,40,3.0,2.11399,0.394737,0.596491,0.798246,1,13270,6713,8028,8549,Unit-specific,Unit-specific,Unit-specific,Unit-specific,210,0.001,0.004,0.02,0.003,0,3.0,89,0.13,0.30,0,0
4,102_CT_1,102,1,U20,CT,Oil CT,Oil,8.0,4.88,1.0467,20.0,8,10,0,1.0,1.0,3.0,1,0.0,0.0,5.0,5.0,5.0,0,0.10,450,50,2.0,10.34940,0.400000,0.600000,0.800000,1,14639,8597,9147,9622,0.2,0.2,0.5,0.036,160,0.002,0.004,0.11,0.040,0,2.8,24,0.13,0.32,0,0


wind_gen_df:


,BusNum,GenUID,Gen ID,Unit Group,Unit Type,Category,GenFuelType,MW Inj,MVAR Inj,V Setpoint p.u.,GenMWMax,PMin MW,QMax MVAR,QMin MVAR,Min Down Time Hr,Min Up Time Hr,Ramp Rate MW/Min,Start Time Cold Hr,Start Time Warm Hr,Start Time Hot Hr,Start Heat Cold MBTU,Start Heat Warm MBTU,Start Heat Hot MBTU,Non Fuel Start Cost $,FOR,MTTF Hr,MTTR Hr,Scheduled Maint Weeks,Fuel Price $/MMBTU,Output_pct_0,Output_pct_1,Output_pct_2,Output_pct_3,HR_avg_0,HR_incr_1,HR_incr_2,HR_incr_3,Fuel Sulfur Content %,Emissions SO2 Lbs/MMBTU,Emissions NOX Lbs/MMBTU,Emissions Part Lbs/MMBTU,Emissions CO2 Lbs/MMBTU,Emissions CH4 Lbs/MMBTU,Emissions N2O Lbs/MMBTU,Emissions CO Lbs/MMBTU,Emissions VOCs Lbs/MMBTU,Damping Ratio,Inertia MJ/MW,Base MVA,Transformer X p.u.,Unit X p.u.,Pump Load MW,Storage Roundtrip Efficiency,Latitude,Longitude
0,320,320_PV_1,1,PV,PV,Solar PV,Wind,0.0,0.0,1.0,51.6,0,0,0,0.0,0.0,51.6,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,34.462466,-117.063508
1,314,314_PV_1,1,PV,PV,Solar PV,Wind,0.0,0.0,1.0,51.6,0,0,0,0.0,0.0,51.6,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,34.944964,-117.684879
2,314,314_PV_2,2,PV,PV,Solar PV,Wind,0.0,0.0,1.0,51.6,0,0,0,0.0,0.0,51.6,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,34.944964,-117.684879
3,313,313_PV_1,1,PV,PV,Solar PV,Wind,0.0,0.0,1.0,95.1,0,0,0,0.0,0.0,95.1,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,34.265268,-118.018855
4,314,314_PV_3,3,PV,PV,Solar PV,Wind,0.0,0.0,1.0,92.7,0,0,0,0.0,0.0,92.7,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,34.944964,-117.684879


<IPython.core.display.Javascript object>

## instantiate Grid class

In [6]:
# to instantiate a grid we need: name, bus, generator, and wind generator dataframes from Parser
# really, we only wind generators, will change in the future
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

Grid(name=RTS, buses=73, generators=158, wind_generators=61, wind_sites=0)


RTS grid info: 

 number of buses: 73
 number of generators: 158
 number of wind generators: 61
 number of solar generators: 0
 total generator capacity: 14549.80 MW
 wind capacity/penetration: 5423.80 MW / 37.28%
 solar capacity/penetration: 0.00 MW / 0.00%


<IPython.core.display.Javascript object>

## (optional) change_wind_penetration method

In [7]:
# ?grid.change_wind_penetration

<IPython.core.display.Javascript object>

## retrieve_wind_sites method

In [8]:
# ?grid.retrieve_wind_sites

<IPython.core.display.Javascript object>

In [9]:
# retrieve wind sites matching current wind penetration

# retrieve wind sites (wind_sites are initially set to empty df )
grid
grid.retrieve_wind_sites()
grid
grid.wind_sites.head()

Grid(name=RTS, buses=73, generators=158, wind_generators=61, wind_sites=0)

Retrieving wind sites ...
Done


Grid(name=RTS, buses=73, generators=158, wind_generators=61, wind_sites=464)

,SiteID,Capacity,Point,Latitude,Longitude,BusNum,GenUID
0,27886,14.0,POINT (-117.061111 34.450134),34.450134,-117.061111,320,320_PV_1
1,27885,14.0,POINT (-117.08252 34.445366),34.445366,-117.082520,320,320_PV_1
2,27719,14.0,POINT (-117.055344 34.432465),34.432465,-117.055344,320,320_PV_1
3,27718,16.0,POINT (-117.076752 34.427704),34.427704,-117.076752,320,320_PV_1
4,32573,10.0,POINT (-117.678467 34.918629),34.918629,-117.678467,314,314_PV_1


<IPython.core.display.Javascript object>

## make tables  

In [10]:
grid.make_tables(
    actuals_start=pd.Timestamp("2007-01-01 00:00:00", tz="utc"),
    actuals_end=pd.Timestamp("2007-12-31 23:55:00", tz="utc"),
    scenarios_start=pd.Timestamp("2008-01-01 00:00:00", tz="utc"),
    scenarios_end=pd.Timestamp("2013-12-31 23:55:00", tz="utc"),
)

Retrieving WTK data ...
Done
Retrieving WTK data ...
Done


<IPython.core.display.Javascript object>

## change actuals year if needed

In [11]:
# for actuals, make year you want
grid.actuals.index = grid.actuals.index.map(lambda t: t.replace(year=2020))
# see what you got
print("\nactuals_df:")
grid.actuals.head()
print("\nscenarios_df:")
grid.scenarios.head()


actuals_df:


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1,TotalPower
IssueTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,7.624867,2.116968,0.000000,13.228901,9.584932,5.468557,27.678979,20.935446,4.275255,52.914559,32.175114,16.346166,43.428635,0.241743,0.509268,0.0,0.0,0.005163,0.005306,0.082441,2.642479,0.0,0.005174,0.010468,0.0,3.679466,5.948414,21.053886,20.311098,10.763757,12.769075,38.577822,20.961372,19.197034,32.182018,19.422803,25.124671,8.625576,5.306221,4.983726,6.338001,5.126045,6.409410,4.674566,6.475319,0.0,0.0,0.078874,0.0,0.002318,0.0,3.784255,0.0,0.0,0.0,0.0,0.0,78.601036,12.449511,506.147898,69.770140,1188.044733
2020-01-01 00:05:00+00:00,8.793432,2.311322,0.003210,13.025394,9.684803,5.418012,27.737229,21.777115,7.298298,56.269944,34.050094,22.064174,43.942613,0.272419,0.825216,0.0,0.0,0.009621,0.008777,0.085880,2.525708,0.0,0.008398,0.013195,0.0,4.131939,5.972320,21.626991,20.215263,10.363748,12.282286,38.443924,20.565082,19.394500,33.161116,19.780040,26.033617,8.792948,5.564064,5.956621,7.426216,5.847023,6.686777,5.709951,7.672921,0.0,0.0,0.079882,0.0,0.001342,0.0,4.487132,0.0,0.0,0.0,0.0,0.0,83.371796,13.253890,527.093036,71.973278,1242.012559
2020-01-01 00:10:00+00:00,10.062529,2.417880,0.007723,12.837683,10.140275,5.453775,27.746646,22.766177,10.748600,59.703261,35.739272,27.610489,43.996929,0.241158,1.156562,0.0,0.0,0.014078,0.012991,0.116442,2.403581,0.0,0.012614,0.016419,0.0,4.811929,5.966486,21.070620,19.552895,9.954058,11.989572,38.127097,20.240237,19.334782,33.701752,20.128823,26.790709,9.057797,5.682285,6.866784,8.533768,6.469398,6.988461,6.624738,8.735803,0.0,0.0,0.078915,0.0,0.000887,0.0,5.256120,0.0,0.0,0.0,0.0,0.0,85.920475,14.088491,547.948202,72.505789,1289.631961
2020-01-01 00:15:00+00:00,11.061709,2.514693,0.009585,12.365985,10.774898,5.550576,27.607677,23.657890,14.069851,61.899525,36.279692,32.096466,44.000000,0.218800,1.473328,0.0,0.0,0.016308,0.015470,0.144585,2.281413,0.0,0.015095,0.017908,0.0,5.403461,5.743405,20.145670,18.807410,9.620720,11.479858,37.591079,19.777146,19.159603,33.891100,19.818353,26.908486,8.781414,5.556908,7.514319,9.246411,6.647529,6.917050,7.288205,9.262521,0.0,0.0,0.079429,0.0,0.000496,0.0,5.758312,0.0,0.0,0.0,0.0,0.0,87.704370,14.695144,567.989286,72.192094,1324.051234
2020-01-01 00:20:00+00:00,11.940349,2.805101,0.009890,11.799855,11.468630,5.594804,27.255826,24.558766,16.612176,64.112613,35.484159,37.198467,44.000000,0.192736,1.693692,0.0,0.0,0.020026,0.019934,0.159775,2.196529,0.0,0.019561,0.020141,0.0,5.888969,5.137275,19.057209,17.774700,9.119162,10.678414,36.806129,19.052771,19.094828,34.055596,19.166929,26.968767,8.485464,5.696144,7.960717,9.881670,6.515105,6.936951,7.651340,9.463083,0.0,0.0,0.081423,0.0,0.000333,0.0,6.068904,0.0,0.0,0.0,0.0,0.0,91.454344,15.380123,587.789462,73.239203,1356.568046



scenarios_df:


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1,TotalPower,Deviation
IssueTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-01-01 00:00:00+00:00,0.0,3.531922,2.243516,0.0,0.007641,0.0,0.0,2.126981,0.688087,1.616444,0.850132,-0.219380,-0.378990,0.0,0.250922,0.0,-0.001941,-0.011200,-0.013058,0.0,-0.036514,0.000000,-0.062884,-0.123793,0.0,0.0,0.0,0.050078,0.572274,0.532375,0.931206,0.663225,1.379854,0.561249,0.894237,0.481982,0.637456,1.077294,0.811130,0.0,0.0,0.0,0.550508,0.130816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.096177,0.743949,7.381553,1.463597,3482.924239,33.426846
2008-01-01 00:05:00+00:00,0.0,3.162277,1.779229,0.0,0.000000,0.0,0.0,1.099578,0.921280,1.218537,-0.095366,-0.000242,-0.590731,0.0,0.432576,0.0,0.038434,0.010163,-0.013059,0.0,0.067550,0.000000,-0.034478,-0.086155,0.0,0.0,0.0,-0.048218,0.588413,0.527197,0.975754,0.617047,0.975086,0.261095,0.559529,0.232802,0.659883,0.140169,0.365765,0.0,0.0,0.0,0.445129,0.092767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.360137,-2.200020,6.576062,0.726162,3516.351085,22.764353
2008-01-01 00:10:00+00:00,0.0,2.750229,1.099202,0.0,0.000000,0.0,0.0,0.363543,0.996593,1.010809,-1.133276,0.114390,-0.699209,0.0,1.282141,0.0,0.022603,0.011648,-0.015036,0.0,0.391293,0.000000,0.061438,-0.010343,0.0,0.0,0.0,-0.196262,0.394932,0.288234,0.562014,0.422211,0.399680,0.020087,0.269170,0.152769,0.727459,-0.448207,0.120442,0.0,0.0,0.0,0.632481,0.050943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.427479,-3.658002,4.610957,0.031029,3539.115438,14.053443
2008-01-01 00:15:00+00:00,0.0,3.254248,1.278273,0.0,0.000000,0.0,0.0,0.711668,0.879005,1.043050,-1.412587,0.115858,-0.397719,0.0,2.630353,0.0,0.000851,0.098074,-0.008090,0.0,0.894007,0.000000,0.039768,-0.002661,0.0,0.0,0.0,-0.145198,0.477658,0.354053,0.723832,0.733189,0.728078,0.508235,0.473411,0.716318,1.339071,-0.551000,0.131542,0.0,0.0,0.0,0.758426,0.070436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.870218,-1.853164,5.985950,-0.353885,3553.168881,24.091267
2008-01-01 00:20:00+00:00,0.0,2.336896,0.969140,0.0,0.000000,0.0,0.0,0.417748,0.860255,0.456226,-1.302932,0.200501,-0.136885,0.0,3.002472,0.0,0.068485,0.118262,-0.011904,0.0,0.910471,0.020112,0.015798,0.038121,0.0,0.0,0.0,-0.226976,0.339366,0.294291,0.420387,0.626189,0.472909,0.337557,0.350824,0.725379,1.131903,-0.873260,-0.405371,0.0,0.0,0.0,0.481069,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.304139,-3.123615,6.802319,-1.165629,3577.260148,17.454249


<IPython.core.display.Javascript object>

## generate_wind_scenarios

In [19]:

# time period for which to generate scenarios

# a few timestamps timestamp
#sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),] 
sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),pd.Timestamp("2020-07-01 00:20:00+0000", tz="UTC")] 

# #range
# sim_timestamps = pd.date_range(
#    start=pd.Timestamp("2020-07-01 00:00:00+0000", tz="UTC"), end=pd.Timestamp("2020-07-01 00:50:00+0000", tz="UTC"), freq="5min"
# )


# other parameters
sampling_method="monte carlo"
#sampling_method="importance"
n_scenarios = 3
n_periods = 2



all_weights_df = pd.DataFrame(index=sim_timestamps,columns=range(1,n_scenarios+1))

# create multiindex df for all generated scenarios
# three arrays for multiindex:
a1 = [x for x in sim_timestamps for k in range(n_scenarios * n_periods)]
a2 = [x for x in range(1, n_scenarios + 1) for k in range(n_periods)] * len(
    sim_timestamps
)
a3 = [
    t + pd.Timedelta("5min") * k
    for t in sim_timestamps
    for k in list(range(n_periods)) * n_scenarios
]

index = pd.MultiIndex.from_arrays([a1,a2,a3],names=['sim_timestamp','scenario_nr','period_timestamp'])
all_scenarios_df = pd.DataFrame(index=index,columns=grid.wind_generators["GenUID"])



for sim_timestamp in sim_timestamps:
    #print("sim_timestamp = {}".format(sim_timestamp))
    random_seed = np.random.randint(2 ** 31 - 1)
    #random_seed = 594081473
    #print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.1, 0.9, 1.0],
        sampling_method=sampling_method,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        #random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    #all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_scenarios_df.loc[sim_timestamp]=scenarios_df

    #all_weights_df=pd.concat([all_weights_df,weights_df])
    all_weights_df.loc[sim_timestamp]=weights_df.loc[sim_timestamp]




all_actuals_df=grid.actuals.loc[sim_timestamps].drop("TotalPower", axis=1).copy()

all_actuals_df
all_scenarios_df
all_weights_df




,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
IssueTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00+00:00,17.772730,43.187182,39.367452,25.308155,86.221077,51.6,32.138778,46.113121,49.7,69.804438,15.592334,51.6,51.0,5.506304,133.992470,26.784092,5.082690,5.045664,5.182326,2.690151,56.734059,4.802985,5.060221,5.325876,4.355086,0.008016,10.433651,37.578887,23.775425,23.222061,24.560665,30.708072,26.324115,21.580692,28.313068,28.493793,38.990402,19.985708,12.063366,10.346116,13.567343,9.337239,6.965034,11.112188,7.952829,1.492059,2.039280,3.081292,1.776888,0.677965,0.601104,5.567361,0.467886,0.401782,0.835430,0.329836,5.418804,134.115394,601.258549,847.0,190.950487
2020-07-01 00:20:00+00:00,18.173115,42.005930,39.317284,25.846124,87.585799,51.6,32.658785,46.441820,49.7,70.584327,16.227496,51.6,51.0,5.355208,137.046729,26.762298,5.089029,5.091152,5.208223,2.408837,57.589272,4.870098,5.147959,5.387816,4.340262,0.001517,9.387169,37.710131,23.690756,23.146248,24.197482,31.028032,26.340611,21.555903,28.321478,28.174631,38.874231,19.858998,11.980738,10.478560,13.800619,9.329806,7.077890,11.057630,8.085764,1.419466,1.887443,3.219483,1.482678,0.683080,0.664414,5.708483,0.437396,0.353257,0.799007,0.348745,6.203544,135.227869,615.587441,847.0,191.486970


GenUID                                                          320_PV_1  \
sim_timestamp             scenario_nr period_timestamp                     
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00  17.3765   
                                      2020-07-01 00:20:00+00:00  17.3999   
                          2           2020-07-01 00:15:00+00:00  17.4047   
                                      2020-07-01 00:20:00+00:00  17.1513   
                          3           2020-07-01 00:15:00+00:00  16.4183   
                                      2020-07-01 00:20:00+00:00  14.2781   
2020-07-01 00:20:00+00:00 1           2020-07-01 00:20:00+00:00  18.2218   
                                      2020-07-01 00:25:00+00:00  19.0912   
                          2           2020-07-01 00:20:00+00:00  17.7411   
                                      2020-07-01 00:25:00+00:00   17.674   
                          3           2020-07-01 00:20:00+00:00  17.7296   
                                      2020-07-01 00:25:00+00:00  17.6119   

GenUID                                                          314_PV_1  \
sim_timestamp             scenario_nr period_timestamp                     
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00   43.429   
                                      2020-07-01 00:20:00+00:00  43.3537   
                          2           2020-07-01 00:15:00+00:00  43.3995   
                                      2020-07-01 00:20:00+00:00   43.245   
                          3           2020-07-01 00:15:00+00:00  43.4974   
                                      2020-07-01 00:20:00+00:00  43.4974   
2020-07-01 00:20:00+00:00 1           2020-07-01 00:20:00+00:00  43.1776   
                                      2020-07-01 00:25:00+00:00  43.1935   
                          2           2020-07-01 00:20:00+00:00  43.4179   
                                      2020-07-01 00:25:00+00:00  43.5513   
                          3           2020-07-01 00:20:00+00:00  42.7734   
                                      2020-07-01 00:25:00+00:00  42.5357   

GenUID                                                          314_PV_2  \
sim_timestamp             scenario_nr period_timestamp                     
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00   39.396   
                                      2020-07-01 00:20:00+00:00   39.396   
                          2           2020-07-01 00:15:00+00:00  39.1114   
                                      2020-07-01 00:20:00+00:00  38.9401   
                          3           2020-07-01 00:15:00+00:00  39.5945   
                                      2020-07-01 00:20:00+00:00  39.7238   
2020-07-01 00:20:00+00:00 1           2020-07-01 00:20:00+00:00  39.3551   
                                      2020-07-01 00:25:00+00:00  39.3457   
                          2           2020-07-01 00:20:00+00:00  39.5971   
                                      2020-07-01 00:25:00+00:00  39.8119   
                          3           2020-07-01 00:20:00+00:00  39.2417   
                                      2020-07-01 00:25:00+00:00  39.0728   

GenUID                                                          313_PV_1  \
sim_timestamp             scenario_nr period_timestamp                     
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00  24.7666   
                                      2020-07-01 00:20:00+00:00  24.8289   
                          2           2020-07-01 00:15:00+00:00  24.4745   
                                      2020-07-01 00:20:00+00:00  24.0059   
                          3           2020-07-01 00:15:00+00:00  24.6887   
                                      2020-07-01 00:20:00+00:00  24.6887   
2020-07-01 00:20:00+00:00 1           2020-07-01 00:20:00+00:00  25.3665   
                                      2020-07-01 00:25:00+00:00  25.1464   
                          2           2020-07

,1,2,3
2020-07-01 00:15:00+00:00,1,1,1
2020-07-01 00:20:00+00:00,1,1,1


<IPython.core.display.Javascript object>

# plot 

In [13]:
# total power
# choose sim_timestamp for which to plot all scenarios
sim_timestamp = sim_timestamps[0]

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)
plot_df = pd.DataFrame(index=timestamps, columns=range(1, n_scenarios + 1),)
for scenario_nr in range(1, n_scenarios + 1):
    s = all_scenarios_df.loc[(sim_timestamp, scenario_nr,)].sum(axis=1)
    s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

    plot_df[scenario_nr] = s

plot_df.iplot()
#plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power", asImage=True,)




<IPython.core.display.Javascript object>

# Importance sampling weights

$E_{f}\left[y\left(X\right)\right]=\int y\left(x\right)f(x)dx=\int y\left(x\right)\frac{f(x)}{g(x)}g(x)dx=E_{g}\left[y\left(X\right)\frac{f(X)}{g(X)}\right]$


$E\left[y\left(X\right)\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;f$


$E\left[y\left(X\right)\frac{f(X)}{g(X)}\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)\frac{f(X_i)}{g(X_i)}} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;g$

$\frac{f(X_i)}{g(X_i)} = w_i \;\; \textrm{are importance sampling weights} $

# aggregate by bus (occasionally needed) 

In [18]:
all_actuals_df = all_actuals_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
all_scenarios_df = all_scenarios_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
all_actuals_df
all_scenarios_df

,101,102,103,104,113,118,119,122,212,213,215,303,308,309,310,312,313,314,317,319,320,324
sim_timestamp,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00,20.234747,10.265016,4.355086,2.690151,5.506304,11.703522,0.008016,190.950487,56.734059,5.418804,26.784092,847.0,10.433651,134.115394,61.705455,69.804438,380.008222,220.375711,601.258549,133.992470,75.655805,152.3
2020-07-01 00:20:00,20.497024,10.297251,4.340262,2.408837,5.355208,11.294968,0.001517,191.486970,57.589272,6.203544,26.762298,847.0,9.387169,135.227869,62.669316,70.584327,380.462038,220.509013,615.587441,137.046729,76.633977,152.3


101        102  \
sim_timestamp       scenario_nr period_timestamp                            
2020-07-01 00:15:00 1           2020-07-01 00:15:00  20.542462  10.461726   
                                2020-07-01 00:20:00  20.841194  10.216850   
                    2           2020-07-01 00:15:00  21.283238  10.681539   
                                2020-07-01 00:20:00  21.951422  11.046175   
                    3           2020-07-01 00:15:00  20.422395  10.396708   
                                2020-07-01 00:20:00  20.422395  10.396708   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  19.982542  10.178371   
                                2020-07-01 00:25:00  19.764285  10.137115   
                    2           2020-07-01 00:20:00  19.652277  10.260496   
                                2020-07-01 00:25:00  19.634760  10.832372   
                    3           2020-07-01 00:20:00  19.224822   9.842349   
                                2020-07-01 00:25:00  18.134565   9.661008   

                                                          103       104  \
sim_timestamp       scenario_nr period_timestamp                          
2020-07-01 00:15:00 1           2020-07-01 00:15:00  4.345550  3.095569   
                                2020-07-01 00:20:00  4.345550  3.202978   
                    2           2020-07-01 00:15:00  5.268250  3.197290   
                                2020-07-01 00:20:00  5.803670  3.358688   
                    3           2020-07-01 00:15:00  4.368187  2.844482   
                                2020-07-01 00:20:00  4.402924  2.714153   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  1.789475  2.931696   
                                2020-07-01 00:25:00  0.000000  3.180082   
                    2           2020-07-01 00:20:00  4.354243  2.295341   
                                2020-07-01 00:25:00  4.357162  2.054715   
                    3           2020-07-01 00:20:00  4.342768  2.505655   
                                2020-07-01 00:25:00  4.332516  2.322906   

                                                          113        118  \
sim_timestamp       scenario_nr period_timestamp                           
2020-07-01 00:15:00 1           2020-07-01 00:15:00  5.913043  12.082735   
                                2020-07-01 00:20:00  6.088026  12.161871   
                    2           2020-07-01 00:15:00  5.912664  10.111864   
                                2020-07-01 00:20:00  5.812822   8.490693   
                    3           2020-07-01 00:15:00  7.401730  11.129427   
                                2020-07-01 00:20:00  9.189573  10.621052   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  4.871606  11.827938   
                                2020-07-01 00:25:00  3.148812  11.519396   
                    2           2020-07-01 00:20:00  4.817238  10.325950   
                                2020-07-01 00:25:00  4.307924   9.283041   
                    3           2020-07-01 00:20:00  5.132796  10.196612   
                                2020-07-01 00:25:00  4.597481   9.088834   

                                                          119         122  \
sim_timestamp       scenario_nr period_timestamp                            
2020-07-01 00:15:00 1           2020-07-01 00:15:00  0.006676  187.355471   
                                2020-07-01 00:20:00  0.006732  183.105345   
                    2           2020-07-01 00:15:00  2.414618  189.225930   
                                2020-07-01 00:20:00  4.006913  188.570136   
                    3           2020-07-01 00:15:00  0.009781  188.805560   
                                2020-07-01 00:20:00  0.009781  187.836718   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  0.000000  185.819496   
                                2020-07-01 00:25:00  0.000000  178.407526   
                    2           2020-07-01 00:20:00  0.000000  188.185072   
                 

<IPython.core.display.Javascript object>

# drop tz  (occasionally needed)

In [15]:
# drop tz from actuals
all_actuals_df.index = all_actuals_df.index.map(lambda t: t.replace(tzinfo=None))

# drop tz from scenarios
all_scenarios_df.index=all_scenarios_df.index.map(
    lambda t: (t[0].replace(tzinfo=None), t[1], t[2].replace(tzinfo=None))
)

all_actuals_df
all_scenarios_df



,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
sim_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00,17.772730,43.187182,39.367452,25.308155,86.221077,51.6,32.138778,46.113121,49.7,69.804438,15.592334,51.6,51.0,5.506304,133.992470,26.784092,5.082690,5.045664,5.182326,2.690151,56.734059,4.802985,5.060221,5.325876,4.355086,0.008016,10.433651,37.578887,23.775425,23.222061,24.560665,30.708072,26.324115,21.580692,28.313068,28.493793,38.990402,19.985708,12.063366,10.346116,13.567343,9.337239,6.965034,11.112188,7.952829,1.492059,2.039280,3.081292,1.776888,0.677965,0.601104,5.567361,0.467886,0.401782,0.835430,0.329836,5.418804,134.115394,601.258549,847.0,190.950487
2020-07-01 00:20:00,18.173115,42.005930,39.317284,25.846124,87.585799,51.6,32.658785,46.441820,49.7,70.584327,16.227496,51.6,51.0,5.355208,137.046729,26.762298,5.089029,5.091152,5.208223,2.408837,57.589272,4.870098,5.147959,5.387816,4.340262,0.001517,9.387169,37.710131,23.690756,23.146248,24.197482,31.028032,26.340611,21.555903,28.321478,28.174631,38.874231,19.858998,11.980738,10.478560,13.800619,9.329806,7.077890,11.057630,8.085764,1.419466,1.887443,3.219483,1.482678,0.683080,0.664414,5.708483,0.437396,0.353257,0.799007,0.348745,6.203544,135.227869,615.587441,847.0,191.486970


GenUID                                              320_PV_1 314_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00   17.352  43.2861   
                                2020-07-01 00:20:00  17.1116   42.884   
                    2           2020-07-01 00:15:00  17.3496  43.4974   
                                2020-07-01 00:20:00  17.3496  43.4974   
                    3           2020-07-01 00:15:00  17.3528  43.3625   
                                2020-07-01 00:20:00  17.3528  43.3049   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  17.2449  44.3629   
                                2020-07-01 00:25:00  16.7961   45.217   
                    2           2020-07-01 00:20:00  17.7727  43.2582   
                                2020-07-01 00:25:00  17.7708   43.253   
                    3           2020-07-01 00:20:00  17.6991  42.0771   
                                2020-07-01 00:25:00  17.8847  41.4884   

GenUID                                              314_PV_2 313_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  38.9437  23.4007   
                                2020-07-01 00:20:00  38.5552  22.0345   
                    2           2020-07-01 00:15:00  39.3971  24.6094   
                                2020-07-01 00:20:00  39.3971  24.4277   
                    3           2020-07-01 00:15:00  39.3918   26.769   
                                2020-07-01 00:20:00  39.3918  28.4886   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  40.7943  28.0169   
                                2020-07-01 00:25:00  42.6974  30.6601   
                    2           2020-07-01 00:20:00  39.3842  25.3671   
                                2020-07-01 00:25:00   39.407   25.702   
                    3           2020-07-01 00:20:00  39.6572  26.4845   
                                2020-07-01 00:25:00  39.9166  27.9697   

GenUID                                              314_PV_3 314_PV_4  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  85.2435     51.6   
                                2020-07-01 00:20:00  84.7942     51.6   
                    2           2020-07-01 00:15:00  85.5737     51.6   
                                2020-07-01 00:20:00   85.579     51.6   
                    3           2020-07-01 00:15:00  85.5677     51.6   
                                2020-07-01 00:20:00  85.5677     51.6   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  85.3649  51.2087   
                                2020-07-01 00:25:00  84.5656  50.8537   
                    2           2020-07-01 00:20:00  85.7486  50.5862   
                                2020-07-01 00:25:00  85.2202  49.4117   
                    3           2020-07-01 00:20:00  83.8939  48.5553   
                                2020-07-01 00:25:00  82.5698  46.2018   

GenUID                                              313_PV_2 310_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  31.0582   45.823   
                                2020-07-01 00:20:00  30.6541   45.805   
                    2           2020-07-01 00:15:00  31.3156  45.8589   
                                2020-07-01 00:20:00  31.1578  45.8595   
                    3           2020-07-01 00:15:00  31.5706  45.8472   
                                2020-07-01 00:20:00  31.6159  45.8436   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  35.3017  44.7327   
                                2020-07-01 00:25:00  38.4049  43.0292   
                    2           2020-07-01 00:20:00  31.8922  46.1131   
                                2020-07-01 00:25:00  31.8456  46.1131   
                    3           2020-07-01 00:20:00  33

<IPython.core.display.Javascript object>

# save as .csv

In [16]:
df = all_scenarios_df.copy()
# if scenarios are single period, we can drop period_timestamp index level
if n_periods == 1:
    df.index=df.index.droplevel("period_timestamp")

filename = './scenarios.csv'
print("\nsaving all_scenarios_df to {}".format(filename))
df
df.to_csv(filename)

# save weights
filename = './weights.csv'
print("\nsaving all_weights_df to {}".format(filename))
all_weights_df
all_weights_df.to_csv(filename)


# take actuals corresponding to scenarios
df=all_actuals_df
df.index=df.index.rename("sim_timestamp")
filename = './actuals.csv'
print("\nsaving actuals to {}".format(filename))
df
df.to_csv(filename)



saving all_scenarios_df to ./scenarios.csv


GenUID                                              320_PV_1 314_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00   17.352  43.2861   
                                2020-07-01 00:20:00  17.1116   42.884   
                    2           2020-07-01 00:15:00  17.3496  43.4974   
                                2020-07-01 00:20:00  17.3496  43.4974   
                    3           2020-07-01 00:15:00  17.3528  43.3625   
                                2020-07-01 00:20:00  17.3528  43.3049   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  17.2449  44.3629   
                                2020-07-01 00:25:00  16.7961   45.217   
                    2           2020-07-01 00:20:00  17.7727  43.2582   
                                2020-07-01 00:25:00  17.7708   43.253   
                    3           2020-07-01 00:20:00  17.6991  42.0771   
                                2020-07-01 00:25:00  17.8847  41.4884   

GenUID                                              314_PV_2 313_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  38.9437  23.4007   
                                2020-07-01 00:20:00  38.5552  22.0345   
                    2           2020-07-01 00:15:00  39.3971  24.6094   
                                2020-07-01 00:20:00  39.3971  24.4277   
                    3           2020-07-01 00:15:00  39.3918   26.769   
                                2020-07-01 00:20:00  39.3918  28.4886   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  40.7943  28.0169   
                                2020-07-01 00:25:00  42.6974  30.6601   
                    2           2020-07-01 00:20:00  39.3842  25.3671   
                                2020-07-01 00:25:00   39.407   25.702   
                    3           2020-07-01 00:20:00  39.6572  26.4845   
                                2020-07-01 00:25:00  39.9166  27.9697   

GenUID                                              314_PV_3 314_PV_4  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  85.2435     51.6   
                                2020-07-01 00:20:00  84.7942     51.6   
                    2           2020-07-01 00:15:00  85.5737     51.6   
                                2020-07-01 00:20:00   85.579     51.6   
                    3           2020-07-01 00:15:00  85.5677     51.6   
                                2020-07-01 00:20:00  85.5677     51.6   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  85.3649  51.2087   
                                2020-07-01 00:25:00  84.5656  50.8537   
                    2           2020-07-01 00:20:00  85.7486  50.5862   
                                2020-07-01 00:25:00  85.2202  49.4117   
                    3           2020-07-01 00:20:00  83.8939  48.5553   
                                2020-07-01 00:25:00  82.5698  46.2018   

GenUID                                              313_PV_2 310_PV_1  \
sim_timestamp       scenario_nr period_timestamp                        
2020-07-01 00:15:00 1           2020-07-01 00:15:00  31.0582   45.823   
                                2020-07-01 00:20:00  30.6541   45.805   
                    2           2020-07-01 00:15:00  31.3156  45.8589   
                                2020-07-01 00:20:00  31.1578  45.8595   
                    3           2020-07-01 00:15:00  31.5706  45.8472   
                                2020-07-01 00:20:00  31.6159  45.8436   
2020-07-01 00:20:00 1           2020-07-01 00:20:00  35.3017  44.7327   
                                2020-07-01 00:25:00  38.4049  43.0292   
                    2           2020-07-01 00:20:00  31.8922  46.1131   
                                2020-07-01 00:25:00  31.8456  46.1131   
                    3           2020-07-01 00:20:00  33


saving all_weights_df to ./weights.csv


,1,2,3
2020-07-01 00:15:00+00:00,1,1,1
2020-07-01 00:20:00+00:00,1,1,1



saving actuals to ./actuals.csv


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
sim_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00,17.772730,43.187182,39.367452,25.308155,86.221077,51.6,32.138778,46.113121,49.7,69.804438,15.592334,51.6,51.0,5.506304,133.992470,26.784092,5.082690,5.045664,5.182326,2.690151,56.734059,4.802985,5.060221,5.325876,4.355086,0.008016,10.433651,37.578887,23.775425,23.222061,24.560665,30.708072,26.324115,21.580692,28.313068,28.493793,38.990402,19.985708,12.063366,10.346116,13.567343,9.337239,6.965034,11.112188,7.952829,1.492059,2.039280,3.081292,1.776888,0.677965,0.601104,5.567361,0.467886,0.401782,0.835430,0.329836,5.418804,134.115394,601.258549,847.0,190.950487
2020-07-01 00:20:00,18.173115,42.005930,39.317284,25.846124,87.585799,51.6,32.658785,46.441820,49.7,70.584327,16.227496,51.6,51.0,5.355208,137.046729,26.762298,5.089029,5.091152,5.208223,2.408837,57.589272,4.870098,5.147959,5.387816,4.340262,0.001517,9.387169,37.710131,23.690756,23.146248,24.197482,31.028032,26.340611,21.555903,28.321478,28.174631,38.874231,19.858998,11.980738,10.478560,13.800619,9.329806,7.077890,11.057630,8.085764,1.419466,1.887443,3.219483,1.482678,0.683080,0.664414,5.708483,0.437396,0.353257,0.799007,0.348745,6.203544,135.227869,615.587441,847.0,191.486970


<IPython.core.display.Javascript object>

# saving as .aux 

In [16]:
!mkdir aux

In [17]:
#save_dir = "/Users/isatkaus/projects/ecp/powerscenarios/data/scenarios-data/tamara/"
save_dir = "./aux/"


###################################### save scenarios
for i in range(len(all_scenarios_df)):

    s = all_scenarios_df.iloc[i]

    # create filename from multiindex, option 1: <simulation timestamp>_<scenario number>_<period timestamp>
    # filename = str(s.name[0]).replace(' ','-')+'_'+str(s.name[1])+'_'+str(s.name[2]).replace(' ','-')

    # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
    filename = (
        str(s.name[0]).replace(" ", "-")
        + "_S"
        + str(s.name[1])
        + "_P"
        + str(((s.name[2] - s.name[0]).seconds // 60) // 5 + 1)
        + ".aux"
    )

    filename = save_dir + filename

    delimiter = " "
    delimiter = "\t"

    with open(filename, "w") as f:
        # .aux header
        _ = f.write("DATA (Gen, [BusNum,GenID,GenMW)]\n")
        _ = f.write("{\n")
        # each series entry is one line
        for k in range(len(s)):
            _ = f.write(
                delimiter
                + s.index[k].split("_")[0]
                + delimiter
                + '"'
                + s.index[k].split("_")[2]
                + '"'
                + delimiter
                + str(s[k])
                + "\n"
            )
        # .aux EOF
        _ = f.write("}\n")


######################################## save actuals

for i in range(len(all_actuals_df)):

    s = all_actuals_df.iloc[i]

    # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
    filename = str(s.name).replace(" ", "-") + "_A" + ".aux"

    filename = save_dir + filename

    delimiter = " "
    delimiter = "\t"

    with open(filename, "w") as f:
        # .aux header
        _ = f.write("DATA (Gen, [BusNum,GenID,GenMW)]\n")
        _ = f.write("{\n")
        # each series entry is one line
        for k in range(len(s)):
            _ = f.write(
                delimiter
                + s.index[k].split("_")[0]
                + delimiter
                + '"'
                + s.index[k].split("_")[2]
                + '"'
                + delimiter
                + str(s[k])
                + "\n"
            )
        # .aux EOF
        _ = f.write("}\n")

# Scratch